In [2]:
!pip install -r requirements.txt 

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of pandas-profiling to determine which version is compatible with other requirements. This could take a while.
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl.metadata (23 kB)
  Using cached pandas_profiling-3.0.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached visions-0.7.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached htmlmin-0.1.12.tar.gz (19 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [27 lines of output]
      Traceback (most recent call last):
        File "/home/leo/Documents/dev/oca-ia/.venv/lib/python3.13/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 389, in <module>
     

In [3]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL
from sklearn.model_selection import KFold
from torch.utils.data import  DataLoader, TensorDataset, Dataset
from torch import nn
import timm
from tqdm.notebook import tqdm
import torch.optim as optim
import os

## Exemplo de classificador com leitura do CSV. 


In [4]:


## Cria funcao para validar se pasta a ser inserida existe. Caso nao exista, cria a pasta
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f'Pasta {folder_path} criada.')
    else:
        print(f'Pasta {folder_path} ja existe.')

In [5]:
## Criacao de pastas output e models se nao existirem
import os
create_folder_if_not_exists('output')
create_folder_if_not_exists('models')

Pasta output ja existe.
Pasta models ja existe.


In [6]:
ds_file = pd.read_csv('dataset/oca.csv')
ds_file = ds_file[['path','oca']]
ds_file.rename(columns={'oca':'label'}, inplace=True)

In [7]:
ds_file.to_csv('dataset/oca_incor.csv', index=False)
ds_file = pd.read_csv('dataset/oca_incor.csv')

## Geracao Tensor com imagens filtradas

In [8]:
## Loop
img_dataset = np.ones((ds_file.shape[0],3,256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i,2)
    j+=1
tensor_imagem = torch.tensor(img_dataset)



## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ OCA, NOCA]

In [9]:
tensor_label = torch.tensor(np.array(ds_file['label'].astype(int)))
tensor_label

tensor([0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

In [10]:
##
ds_file.columns
labels = [ 'label']
ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_file['label'].astype(int)))
tensor_label

tensor([0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1])

### Resumo:

model(tensor_imagem,ds_id_sex_et)

labels : tensor_label

## Criacao Subset 



## K-fold = 10


In [11]:
kf = KFold(n_splits=5)
kf.get_n_splits(tensor_imagem)
print(kf)
#for i in enumerate(kf.split(tensor_imagem)):
#    print(i)

KFold(n_splits=5, random_state=None, shuffle=False)


## Criacao do dataset de treino e validacao com batchsize

In [12]:
BATCH_SIZE = 16
train_dataset = TensorDataset(tensor_imagem, tensor_label)
train_loader_img = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
train_dataset[0][0]

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

## Create a model to concat layers

O modelo utilizado será o mesmo do artigo: Artificial Intelligence-Driven Screening System for Rapid Image-Based Classification of 12-Lead ECG Exams: A Promising Solution for Emergency Room Prioritization 

Desenvolvido conforme a figura abaixo: 

![image.png](attachment:image.png)

In [14]:
#from models import ECGClassifierResnet

class ECGClassifierResnet(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True) 
        self.base_model=timm.create_model('resnet50d.ra4_e3600_r224_in1k',pretrained=True)
        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=5,pretrained=True)


        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 2048        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size,1)
        ) # saida como Sigmoid multilabel

    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        #output = nn.Softmax(dim=1)(x)
        return output

## Train - Valid Loop

Fluxo de treinamento

Input: modelo, dataloader_train, dataloader_teste , epocas.
- Para cada epoca

    - treinamento:

        - para cada batch (lote de imagens) no dataloader_train: (executo para todo o dataloader)
            
            leitura das imagens e rotulos. 
            
            prediz o rotulo (outputs): rotulos preditos para o tamanho do batch (ex: para um batch de 10, o 
            output tem tamanho 10)
            
            calcula o Loss () 
            
            aplica o backward -> Ajuste dos pesos nos neuronios de acordo com o valor de loss (w_{i})

            otimizo o modelo com optimizer.step()

    - validacao (avaliacao do desempenho para aquele conjunto de treinamento):
        - para cada batch (lote de imagens) no dataloader_train: (executo para todo o dataloader)

            leitura das imagens e rotulos.

            prediz o rotulo (outputs): rotulos preditos para o tamanho do batch (ex: para um batch de 10, o 
            output tem tamanho 10)
            
            calcula o Loss ()


    - Calculo de metricas de loss media de validacao.

    - Calculo de metrica de loss media de treinamento. 

    - Print de metricas de desempenho para a epoca. 








In [15]:
def salvar_model(model, path = 'output/modelos', name_file='model.pth'):
    full_path = os.path.join(path, name_file)
    torch.save(model.state_dict(), full_path)

In [16]:

def salvar_metricas(path, name_file_train='train_loss_total.npy', name_file_val='val_loss_total.npy',predict_label=None, true_label=None):
    create_folder_if_not_exists(path)
    full_path_train = os.path.join(path, name_file_train)
    full_path_val = os.path.join(path, name_file_val)
    np.save(full_path_train, np.array(predict_label))
    np.save(full_path_val, np.array(true_label))
    print(f'Metricas salvas em {path} com os nomes {name_file_train} e {name_file_val} e tamanhos {np.array(predict_label).shape} e {np.array(true_label).shape}')


In [17]:
from torch.utils.data import  Dataset

class Subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    @property
    def classes(self):
        return self.data.classes

    def shape(self):
        return self.dataset

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def simple_loop(model, train_image, val_image, epochs, batch_size, fold_index):
    # Simple training loop
    num_epochs = epochs
    train_losses, val_losses = [], []
    lim_loss = 1.5
    iter_size = batch_size
    print(f'Number of training images per iteration: {iter_size}')
    #model = modelo( num_classes=5)
    model.to(device)
    criterion =  nn.L1Loss()
    #criterion =  nn.CrossEntropyLoss()
    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    predict_label_full_train = []
    predict_label_full = []
    true_label_full_train = []
    true_label_full = []
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        predict_label_train =[]
        true_label_train =[]
        running_loss_train = 0.0
        for images, labels in tqdm(train_image, desc='Training loop'):
            # Move inputs and labels to the device
            images = images.to(torch.float)
            image, label = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(image)
            #print(outputs)
            #print(label)
            loss_train = criterion(outputs.float(), label.float())
            loss_train.backward()
            optimizer.step()
            running_loss_train += loss_train.item() * label.size(0)
            try:
                _pred_train = outputs.cpu().data.numpy().astype(int).T[0].tolist()
                if(len(_pred_train) == iter_size):
                    predict_label_train.append(_pred_train)
                    _true_train = label.cpu().data.numpy().astype(int).T.tolist()
                    true_label_train.append(_true_train)

            except Exception as e:
                print(f"Concatenation error iter: {e}")
                print(_pred_train)
                print(predict_label_train)    
        train_loss = running_loss_train / len(train_image.dataset)
        train_losses.append(train_loss)
        try:
            _p_train = predict_label_train
            _t_train = true_label_train
            predict_label_full_train.append(_p_train)
            true_label_full_train.append(_t_train)
        except Exception as e:
            print(f"Concatenation error full: {e}")
            print(predict_label_train)
            print(true_label_train)
        model.eval()
        running_loss_valid = 0.0
        rotulos =[] 
        predict_label =[]
        true_label =[]
        _iter=0
        with torch.no_grad():
            for images, labels in tqdm(val_image, desc='Validation loop'):
                # Move inputs and labels to the device
                images = images.to(torch.float)
                images, label = images.to(device), labels.to(device)
                rotulos.append(label.cpu().data.numpy())
                outputs = model(images)
                loss_valid = criterion(outputs.float(), label.float())
                #print( [outputs.cpu().data.numpy().astype(int).T[0]])
                #print(label.cpu().data.numpy().astype(int).T[0])
                #print(predict_label)
                try:
                    _pred = outputs.cpu().data.numpy().astype(int).T[0].tolist()

                    if(len(_pred) == iter_size):
                        predict_label.append(_pred)
                        _true = label.cpu().data.numpy().astype(int).T.tolist()
                        true_label.append(_true)
                except Exception as e:
                    print(f"Concatenation error iter: {e}")
                    print(_pred)
                    print(predict_label)    
                running_loss_valid += loss_valid.item() * label.size(0)
                _iter +=1
        val_loss = running_loss_valid / len(val_image.dataset)
        val_losses.append(val_loss)
        print(f'End validation for epoch {epoch}')
        print(f'Amount of images validated: {val_image}')
        print(f'Label predict shape : {len(predict_label)} for epoch {epoch}')
        print(f'Count of iterations: {_iter} for epoch {epoch}')
        try:
            _p = predict_label
            _t = true_label
            predict_label_full.append(_p)
            true_label_full.append(_t)
        except Exception as e:
            print(f"Concatenation error full: {e}")
            print(predict_label)
            print(true_label)
        #val_acc = accuracy_score(rotulos,output_model)
        print(f'Val accuracy {epoch}:')
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")

    predict_label_full_out = np.array(predict_label_full)
    true_label_full_out = np.array(true_label_full)
    predict_label_full_train_out = np.array(predict_label_full_train)
    true_label_full_train_out = np.array(true_label_full_train)
    print(f'Salvado das metricas de validacao e treino')
    salvar_metricas(path=f'output/metricas/valid/fold_{fold_index}', name_file_train=f'predict_label_valid_fold_{fold_index}.npy', name_file_val=f'true_label_valid_fold_{fold_index}.npy', predict_label= predict_label_full_out, true_label= true_label_full_out)
    salvar_metricas(path=f'output/metricas/train/fold_{fold_index}', name_file_train=f'predict_label_train_fold_{fold_index}.npy', name_file_val=f'true_label_train_fold_{fold_index}.npy', predict_label= predict_label_full_train_out, true_label= true_label_full_train_out)
    print(f'Finalizado o salvamento das metricas')
    return train_losses, val_losses, model,predict_label_full_out, true_label_full_out

In [20]:

train_loss_total = []
val_loss_total =[]
all_models =[]
epochs = 10
BATCH_SIZE = 10
## create first model.
for i, (train_index, test_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")
    ## init train test for folder
    train_dataset_part = Subset( train_dataset, train_index)
    val_dataset_part = Subset( train_dataset, test_index)

    train_loader_img = DataLoader(train_dataset_part, batch_size=BATCH_SIZE, shuffle=True)
    val_loader_img = DataLoader(val_dataset_part, batch_size=BATCH_SIZE, shuffle=True)

    model= ECGClassifierResnet( num_classes=1)
    if(False):
        salvar_model(model, path='output/modelos', name_file=f'model_fold_{i}.pth')
    print(f'Train and valid for Fold {i}')
    t, l,_,outputs,labels = simple_loop(model, train_loader_img,val_loader_img, epochs, batch_size=BATCH_SIZE, fold_index=i)
    ## Evaluate model.
    train_loss_total.append(t)
    val_loss_total.append(l)

Fold 0:
Train and valid for Fold 0
Number of training images per iteration: 10


Training loop:   0%|          | 0/18 [00:00<?, ?it/s]

/home/leo/Documents/dev/oca-ia/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/home/leo/Documents/dev/oca-ia/.venv/lib/python3.13/site-packages/torch/nn/modules/loss.py:129: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Validation loop:   0%|          | 0/5 [00:00<?, ?it/s]

End validation for epoch 0
Amount of images validated: <torch.utils.data.dataloader.DataLoader object at 0x72a91c49d150>
Label predict shape : 4 for epoch 0
Count of iterations: 5 for epoch 0
Val accuracy 0:
Epoch 1/10 - Train loss: 0.6862721179806909, Validation loss: 7750673.545454546


...

Validation loop:   0%|          | 0/5 [00:00<?, ?it/s]

End validation for epoch 8
Amount of images validated: <torch.utils.data.dataloader.DataLoader object at 0x72a91c49c550>
Label predict shape : 4 for epoch 8
Count of iterations: 5 for epoch 8
Val accuracy 8:
Epoch 9/10 - Train loss: 0.5017026456105227, Validation loss: 0.550002820270006


Training loop:   0%|          | 0/18 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/5 [00:00<?, ?it/s]

End validation for epoch 9
Amount of images validated: <torch.utils.data.dataloader.DataLoader object at 0x72a91c49c550>
Label predict shape : 4 for epoch 9
Count of iterations: 5 for epoch 9
Val accuracy 9:
Epoch 10/10 - Train loss: 0.5054807201286272, Validation loss: 0.532600158868834
Salvado das metricas de validacao e treino
Pasta output/metricas/valid/fold_4 ja existe.
Metricas salvas em output/metricas/valid/fold_4 com os nomes predict_label_valid_fold_4.npy e true_label_valid_fold_4.npy e tamanhos (10, 4, 10) e (10, 4, 10)
Pasta output/metricas/train/fold_4 ja existe.
Metricas salvas em output/metricas/train/fold_4 com os nomes predict_label_train_fold_4.npy e true_label_train_fold_4.npy e tamanhos (10, 17, 10) e (10, 17, 10)
Finalizado o salvamento das metricas


In [ ]:
outputs
## formato ddos outputs [epoch][batch][labels]
#out = np.array(outputs)

array([[[2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754],
        [2754, 2754, 2754, 2754, 2754]],

       [[   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4],
        [   4,    4,    4,    4,    4]],

       [[   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0]],

       [[   0,    0,    0,    0,  

In [ ]:
np.array(labels[0][0:-2])

array([[1, 1, 1, 1, 1],
       [1, 0, 1, 0, 0],
       [0, 0, 0, 1, 0],
       [1, 0, 1, 1, 1],
       [0, 1, 0, 1, 0],
       [1, 1, 0, 0, 1]])

## Métricas de Avaliacao dos modelos

Acuracia

Precisao

Sensibilidade

Especificidade

F1-Score

In [ ]:
#### Leitura de métricas salvas
import numpy as np
predict_label_0 = np.load('/Users/leonardocipriani/Documents/dev/python/Artificial Intelligence Projects/oca-ia/incor_env/output/metricas/train/fold_0/predict_label_train_fold_0.npy')
true_label_0 = np.load('/Users/leonardocipriani/Documents/dev/python/Artificial Intelligence Projects/oca-ia/incor_env/output/metricas/train/fold_0/true_label_train_fold_0.npy')
# --- IGNORE ---
print(true_label_0)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/leonardocipriani/Documents/dev/python/Artificial Intelligence Projects/oca-ia/incor_env/output/metricas/train/fold_0/predict_label_train_fold_0.npy'

In [ ]:
predict_label_0


array([[[0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [4, 4, 4, 4, 4],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1]],

       [[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1]

In [ ]:
true_label_0

array([[1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1],
       [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0]])

In [ ]:
#### Leitura de métricas salvas
import numpy as np
predict_label_0_valid= np.load('/Users/leonardocipriani/Documents/dev/python/Artificial Intelligence Projects/oca-ia/incor_env/output/metricas/valid/fold_0/predict_label_valid_fold_0.npy')
true_label_0_valid= np.load('/Users/leonardocipriani/Documents/dev/python/Artificial Intelligence Projects/oca-ia/incor_env/output/metricas/valid/fold_0/true_label_valid_fold_0.npy')
# --- IGNORE ---
print(predict_label_0_valid)
print(true_label_0_valid)

[[[2821440 2821440 2821440 2821440 2821440]
  [3546063 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2821440 4119212]
  [2821440 2821440 3188174 2821440 2821440]
  [2821440 2821440 2821440 2821440 2471002]
  [2821440 2821440 2821440 2821440 2821440]
  [3436914 2821440 2821440 2821440 3913112]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2821440 3827248 2821440 2821440]
  [2821440 2821440 2631008 2821440 2821440]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2514506 2821440]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2847649 2821440 3869186 2821440]
  [2759107 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2821440 2821440]
  [2821440 2821440 2821440 2821440 3113366]
  [2821440 2821440 2821440 2821440 3379505]]

 [[    564     564    1204     564     564]
  [    564     564     564    

## Criacao graficos de treinamento e validacao

In [ ]:
## Criar graficos de treinamento e validacao
import matplotlib.pyplot as plt
for i in range(5):
    plt.figure(figsize=(10,5))
    plt.plot(train_loss_total[i], label='Train Loss')
    plt.plot(val_loss_total[i], label='Validation Loss')
    plt.title(f'Fold {i} - Train and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(f'output/graficos/loss_fold_{i}.png')
    plt.show()
    plt.close()


In [ ]:
activate_script = os.path.join("incor_env", "Scripts", "activate")
if os.name == 'nt':  # Windows
    activate_command = f"{activate_script}"
else:  # macOS/Linux
    activate_command = f"source {activate_script}"
print(activate_command)

source incor_env/Scripts/activate


In [ ]:

def config_venv():
    print("Criando ambiente virtual...")
    os.system("ls -la")    
    os.system("python -m venv .incor_env")

config_venv()

Criando ambiente virtual...
total 1680
drwxr-xr-x@ 19 leonardocipriani  staff     608 Dec 14 10:14 .
drwxr-xr-x@ 16 leonardocipriani  staff     512 Nov 25 13:15 ..
drwxr-xr-x@ 16 leonardocipriani  staff     512 Dec 14 10:01 .git
-rw-r--r--   1 leonardocipriani  staff      13 Sep 13 17:55 .gitignore
drwxr-xr-x   7 leonardocipriani  staff     224 Dec 14 10:14 .incor_env
drwxr-xr-x   4 leonardocipriani  staff     128 Dec 13 16:10 .ipynb_checkpoints
drwxr-xr-x   9 leonardocipriani  staff     288 Oct 14 20:26 .venv
-rw-r--r--   1 leonardocipriani  staff    1967 Dec 11 07:20 README.md
-rw-r--r--   1 leonardocipriani  staff  683671 Dec  7 10:06 classificador_ml.ipynb
-rw-r--r--   1 leonardocipriani  staff   70005 Dec 14 10:15 classificador_oca copy.ipynb
-rw-r--r--   1 leonardocipriani  staff   84135 Dec 13 16:18 classificador_oca.ipynb
drwxr-xr-x   9 leonardocipriani  staff     288 Dec 13 16:18 dataset
-rw-r--r--@  1 leonardocipriani  staff     632 Sep  3 16:31 exemplo_csv.csv
drwxr-xr-x   3

In [ ]:
import pandas as pd
ds_file = pd.read_csv('dataset/oca_incor.csv')

In [ ]:
ds_file.rename(columns={ds_file.columns[0]:'label'}, inplace=True)

In [ ]:
ds_file

,label,label
0,image1.png,0
1,image2.png,0
2,image3.png,1
3,image4.png,1
4,image5.png,1
...,...,...
211,image16.png,1
212,image16.png,0
213,image16.png,0
214,image16.png,1
